In [1]:
import pandas as pd

In [2]:
# Definindo tipagem de principais colunas
dict_type_cnpj = {
    'cnpj':'str',
    'endereco_cep':'str',
    'cd_cnae_principal':'float',
#     'optante_simples':'bool'
}

## tratamento df_empresas

In [3]:
# Lendo df
df_empresas = pd.read_csv('../raw_data/df_empresas.csv',
                          sep=';',dtype=dict_type_cnpj)

In [4]:
# Renomeando coluna para manter padrão
df_empresas.rename(columns={
    'matriz.empresaMatriz':'empresa_matriz'
},inplace=True)

In [5]:
# Preenchendo valores vazios coluna cnae
df_empresas['cd_cnae_principal'] = df_empresas['cd_cnae_principal'].fillna(0).astype(int)

In [6]:
# Selecionando colunas específicas
df_empresas_essencial = df_empresas[[
    'cnpj', 'dt_abertura', 'empresa_matriz', 'cd_cnae_principal',
    'endereco_cep', 'situacao_cadastral'
]]

In [7]:
# Carga em dbt/seed
df_empresas_essencial.to_csv('../../neoway_case/seeds/empresas_geral.csv', index=False, sep=',')

#### Subdividr o dataframe original nas tabelas dimensão 

- dimensão CEP
- dimensão CNAE

In [8]:
# dimensao cep
df_ceps = df_empresas[[
    'endereco_cep','endereco_municipio', 'endereco_uf', 
    'endereco_regiao','endereco_mesorregiao',
]]

# Drop de duplicatas
df_ceps = df_ceps.drop_duplicates()

# carga em dbt/seed
df_ceps.to_csv('../../neoway_case/seeds/dim_ceps.csv', index=False, sep=',')

In [9]:
# dimesao CNAE
df_cnaes = df_empresas[[
    'cd_cnae_principal','de_cnae_principal',
    'de_ramo_atividade', 'de_setor'
]]

# Tratamento básico em alguns valores com padrão distinto
df_cnaes.loc[:, 'de_cnae_principal'] = df_cnaes['de_cnae_principal'].str.replace(r'\s+DISPENSADA$', 
                                                                                 '', 
                                                                                 regex=True)

# Drop de linhas duplicadas
df_cnaes = df_cnaes.drop_duplicates()

# Preenchendo valores NaN com uma descrição
df_cnaes.loc[:, 'de_ramo_atividade'] = df_cnaes['de_ramo_atividade'].fillna('RAMO NAO INFORMADO')
df_cnaes.loc[:, 'de_setor'] = df_cnaes['de_setor'].fillna('SETOR NAO INFORMADO')

# carga em dbt/seed
df_cnaes.to_csv('../../neoway_case/seeds/dim_cnaes.csv', index=False, sep=',')

## tratamento outros arquivos CSV

In [10]:
# Lendo os arquivos brutos
df_empresas_nivel_atividade = pd.read_csv('../raw_data/empresas_nivel_atividade.csv',
                                          sep=';',dtype=dict_type_cnpj)
df_empresas_porte = pd.read_csv('../raw_data/empresas_porte.csv',
                                sep=';',dtype=dict_type_cnpj)
df_empresas_saude_tributaria = pd.read_csv('../raw_data/empresas_saude_tributaria.csv',
                                           sep=';',dtype=dict_type_cnpj)
df_empresas_simples = pd.read_csv('../raw_data/empresas_simples.csv',
                                  sep=';',dtype=dict_type_cnpj)

In [11]:
# Convertendo valores das colunas optantes_simples e optante_simei de bool para str
df_empresas_simples['optante_simples'] = df_empresas_simples['optante_simples'].map({True: 'Sim', False: 'Não'})
df_empresas_simples['optante_simei'] = df_empresas_simples['optante_simei'].map({True: 'Sim', False: 'Não'})

# Preenchendo valores vazios das colunas optante_simples e optante_simei
df_empresas_simples.optante_simples = df_empresas_simples.optante_simples.fillna('Não informado')
df_empresas_simples.optante_simei = df_empresas_simples.optante_simei.fillna('Não informado')

In [12]:
# Drop de CNPJs que não possuem informações de nivel_atividade e saude_tributaria para reduzir tamanho
df_empresas_nivel_atividade = df_empresas_nivel_atividade.dropna(subset='nivel_atividade')
df_empresas_saude_tributaria = df_empresas_saude_tributaria.dropna(subset='saude_tributaria')

In [13]:
# carga em dbt/seed
df_empresas_nivel_atividade.to_csv('../../neoway_case/seeds/empresas_nivel_atividade.csv',index=False,sep=',')
df_empresas_porte.to_csv('../../neoway_case/seeds/empresas_porte.csv',index=False,sep=',')
df_empresas_saude_tributaria.to_csv('../../neoway_case/seeds/empresas_saude_tributaria.csv',index=False,sep=',')
df_empresas_simples.to_csv('../../neoway_case/seeds/empresas_simples.csv',index=False,sep=',')